**Purpose**

The purpose of this notebook is to analyze *L. crispatus* genomes from NCBI in order to make a phylogentic tree of the strains and also to scan them for NanA, among other potentially interesting proteins.

In [1]:
from elliot_utils import *

In [2]:
analysisPath = Path.cwd().joinpath('analysis_files/glycans_metabolism/')

In [3]:
# object to store information on a strain of L. crispatus
class Strain:
    def __init__(self, csvRow):
        self.name = csvRow['Organism Scientific Name']
        self.id = csvRow['Strain']
        self.accession = csvRow['Assembly Accession']
        self.cpn60 = ''
        self.source = csvRow['Source']
        self.aaSeqs = set()

# protein object for this data
class GlycanAnalysisProtein:
    def __init__(self, entry):
        self.id = self.extractProteinID(entry)
        self.taxa = set()
        self.name = self.extractProteinName(entry)
        self.sequence = entry[entry.find('\n') + 1:].replace('\n', '')

    # This function takes a single protein entry from a .fasta file and returns the protein identifier for that entry.
    def extractProteinID(self, entry):
        stringEnd = entry.find(' ')
        return entry[0:stringEnd]

    # This function takes a single protein entry from a .fasta file and returns the protein identifier for that entry.
    def extractProteinName(self, entry):
        stringStart = entry.find(' ') + 1
        stringEnd = entry.find(' [')
        return entry[stringStart:stringEnd]
    
    # Adds the specified taxa name to this protein's list of taxa, assuming the new taxa is not already in it
    def addTaxa(self, taxon):
        self.taxa.add(taxon)
    
    # Get the fasta entry for this protein, including all taxa that encode it
    def getEntry(self):
        taxaList = []
        for t in self.taxa:
            taxaList.append(t)
        taxaEntry = taxaList[0]
        if len(taxaList) > 1:
            for i in range(1, len(self.taxa)):
                taxaEntry = f'{taxaEntry};{taxaList[i]}'
        return f'>{self.id} {self.name} OS={taxaEntry}\n{self.sequence}\n'

In [4]:
# Returns the amino acid or nucleotide sequence from a fasta entry with no newline characters
def extractSequence(fastaEntry):
    return fastaEntry[fastaEntry.find('\n') + 1:].replace('\n', '')

# Returns a list of fasta entries that can then be parsed
def parseFasta(dbFile):
    toReturn = []
    with open(dbFile, 'r') as database:
        rawText = database.read()
        dataList = rawText.split('\n>')
        dataList[0] = dataList[0][1:]
        del rawText
        for sequence in dataList:
            toReturn.append(sequence)
    return toReturn

# Returns a list containing the protein IDs of homologs in the given file. ID's must be in the second column of the file.
def loadHomologs(homologsFile):
    ids = []
    with homologsFile.open(mode='r', encoding='utf-8-sig') as infile:
        reader = csv.reader(infile)
        for row in reader:
            ids.append(row[1])
    return ids

# Takes in a dictionary in the format {'protName':[List of prot IDs]} and combines the proteins specified in listOldProts
# under newProtName. Returns it as a new dictionary.
def combineHomologs(homologDict, newProtName, listOldProts):
    tempDict = {newProtName:[]} #key=protein name, value=list of homologous protein IDs
    for protName, homologList in homologDict.items():
        if not protName in listOldProts:
            tempDict[protName] = homologList
        else:
            for homolog in homologList:
                tempDict[newProtName].append(homolog)
    return tempDict

In [5]:
# Objects to record data for all species. These will be used to make a supplementary table containing the protein IDs for homologs of each protein for different isolates
fullProtList = ['Strain', 'nanT', 'nanA', 'nanE-II', 'nanK', 'nanE', 'nagA', 'nagB', 'nagE/K', 'galK', 'galT', 'galE', 'pgmA', 'lacE', 'lacB', 'agaD/K', 'agaA', 'agaS', 'pfkA', 'kbaY', 'fucI', 'fucK', 'fucA', 'fucA1', 'fucB', 'fucD', 'fucH', 'fucO']
suppDataHolder = []

# Returns a new list to add to the suppDataHolder for an individual species
def generateEmptyForSupp(strain):
    toReturn = [strain]
    for x in range(len(fullProtList) - 1):
        toReturn.append('')
    return toReturn

In [6]:
# First, read through the metadata file and initialize all of the L. crispatus strains
strainDict = {} #key=Accession number, value=Strain object
metadata = pd.read_csv(analysisPath.joinpath('lcrispatus/strain_metadata.csv'))
for index, row in metadata.iterrows():
    strainDict[row['Assembly Accession']] = Strain(row)

In [7]:
# Go through all the sequences for each strain. Pull out all of the proteins so I can make a BLAST-able database that retains strain information, and retrieve the cpn60 sequence for each strain.
proteinDict = {} # key=amino acid sequence, value=protein object
for assemblyFolder in analysisPath.joinpath('lcrispatus/sequence_data/').iterdir():
    currentStrain = strainDict[assemblyFolder.name]
    for file in assemblyFolder.iterdir():
        if file.suffix == '.fna':
            genes = parseFasta(file)
            cpn60Found = False
            for gene in genes:
                if gene.lower().find('groel') != -1 or gene.lower().find('grol') != -1 or gene.lower().find('60 kda chaperonin') != -1:
                    cpn60Found = True
                    currentStrain.cpn60 = gene[gene.find('\n') + 1:].replace('\n', '')
            if not cpn60Found:
                print(currentStrain.id)
        elif file.suffix == '.faa':
            proteins = parseFasta(file)
            for prot in proteins:
                newProt = GlycanAnalysisProtein(prot)
                if not newProt.sequence in proteinDict.keys():
                    proteinDict[newProt.sequence] = newProt
                proteinDict[newProt.sequence].addTaxa(currentStrain.id)
                currentStrain.aaSeqs.add(newProt.sequence)

In [8]:
# Write out deduplicated proteins
#toWrite = []
#for prot in proteinDict.values():
#    toWrite.append(prot.getEntry())
#with analysisPath.joinpath('lcrispatus/deduplicated_Lcrispatus_proteins.fasta').open(mode='w', newline='') as outfile:
#    outfile.write(''.join(toWrite))

In [9]:
# Write out all the cpn60 sequences. I'll have to manually add the ones for MV-1A-US, FB049-03, FB077-07, and EM-LC1
#toWrite = []
#for strain in strainDict.values():
#    toWrite.append(f'>{strain.id} cpn60\n{strain.cpn60}\n')
#with analysisPath.joinpath('cpn60_sequences.fasta').open(mode='w', newline='') as outfile:
#    outfile.write(''.join(toWrite))

In [10]:
# I made the L. crispatus tree and organized them. Now I'll read back in the order in which the isolates appear on the tree
order = []
with analysisPath.joinpath('lcrispatus/tree_order.csv').open(mode='r', encoding='utf-8-sig') as infile:
    reader = csv.reader(infile)
    for row in reader:
        order.append(row[0])

In [11]:
# I performed BLASTp searches to identify homologs of metabolic proteins in the L. crispatus genomes. Load in those lists.
filesDict = {
    'nanT':analysisPath.joinpath('lcrispatus/homologs/homologs_nanT.csv'),
    'nanA':analysisPath.joinpath('lcrispatus/homologs/homologs_nanA.csv'),
    'nanK':analysisPath.joinpath('lcrispatus/homologs/homologs_nanK.csv'),
    'nanE':analysisPath.joinpath('lcrispatus/homologs/homologs_nanE.csv'),
    'nagA':analysisPath.joinpath('lcrispatus/homologs/homologs_nagA.csv'),
    'nagB':analysisPath.joinpath('lcrispatus/homologs/homologs_nagB.csv'),
    'nagE':analysisPath.joinpath('lcrispatus/homologs/homologs_nagE.csv'),
    'nagK':analysisPath.joinpath('lcrispatus/homologs/homologs_nagK.csv'),
    'galK':analysisPath.joinpath('lcrispatus/homologs/homologs_galK.csv'),
    'galT':analysisPath.joinpath('lcrispatus/homologs/homologs_galT.csv'),
    'galE':analysisPath.joinpath('lcrispatus/homologs/homologs_galE.csv'),
    'pgmA':analysisPath.joinpath('lcrispatus/homologs/homologs_pgmA.csv'),
    'agaK':analysisPath.joinpath('lcrispatus/homologs/homologs_agaK.csv'),
    'agaD':analysisPath.joinpath('lcrispatus/homologs/homologs_agaD.csv'),
    'agaA':analysisPath.joinpath('lcrispatus/homologs/homologs_agaA.csv'),
    'agaS':analysisPath.joinpath('lcrispatus/homologs/homologs_agaS.csv'),
    'pfkA':analysisPath.joinpath('lcrispatus/homologs/homologs_pfkA.csv'),
    'kbaY':analysisPath.joinpath('lcrispatus/homologs/homologs_kbaY.csv'),
    'fucI':analysisPath.joinpath('lcrispatus/homologs/homologs_fucI.csv'),
    'fucK':analysisPath.joinpath('lcrispatus/homologs/homologs_fucK.csv'),
    'fucA':analysisPath.joinpath('lcrispatus/homologs/homologs_fucA.csv')
}
homologsDict = {} #key=protein name, value=list of homologous protein IDs
for proteinName, filePath in filesDict.items():
    homologsDict[proteinName] = loadHomologs(filePath)
homologsDict = combineHomologs(homologsDict, 'nagE/K', ['nagE', 'nagK'])
homologsDict = combineHomologs(homologsDict, 'agaD/K', ['agaD', 'agaK'])
protOrder = ['nanT', 'nanA', 'nanK', 'nanE', 'nagA', 'nagB', 'nagE/K', 'galK', 'galT', 'galE', 'pgmA', 'agaD/K', 'agaA', 'agaS', 'pfkA', 'kbaY', 'fucI', 'fucK', 'fucA']

In [12]:
# Get the amino acid sequences of all the protein homologs so I can look for them in the L. crispatus strains
aaDict = {}
for proteinName in homologsDict.keys():
    aaDict[proteinName] = set()
for protein in proteinDict.values():
    for key, homologList in homologsDict.items():
        if protein.id in homologsDict[key]:
            aaDict[key].add(protein.sequence)

In [13]:
# Get whether the isolates have one of each homolog, in the tree order
homologStatus = {'strain':[]}
for proteinName in protOrder:
    homologStatus[proteinName] = []
for strain in order:
    for lCrispatus in strainDict.values():
        if lCrispatus.id == strain:
            listForStrain = generateEmptyForSupp(lCrispatus.id)
            homologStatus['strain'].append(strain)
            for proteinName in homologsDict.keys():
                protPresent = '-'
                for seq in aaDict[proteinName]:
                    if seq in lCrispatus.aaSeqs:
                        protPresent = '+'
                        listForStrain[fullProtList.index(proteinName)] = proteinDict[seq].id
                homologStatus[proteinName].append(protPresent)
            suppDataHolder.append(listForStrain)

In [65]:
pd.DataFrame(homologStatus).to_csv(analysisPath.joinpath('lcrispatus/Lcrispatus_homolog_summary_v2.csv'), index=False)

In [14]:
### Repeat the analysis for Gardnerella proteins
# First, read through the metadata file and initialize all of the Gardnerella strains
strainDict = {} #key=Accession number, value=Strain object
metadata = pd.read_csv(analysisPath.joinpath('gardnerella/gardnerella_metadata.csv'))
for index, row in metadata.iterrows():
    strainDict[row['Assembly Accession']] = Strain(row)

In [15]:
# Go through all the sequences for each strain. Pull out all of the proteins so I can make a BLAST-able database that retains strain information, and retrieve the cpn60 sequence for each strain.
proteinDict = {} # key=amino acid sequence, value=protein object
for assemblyFolder in analysisPath.joinpath('gardnerella/sequence_data/').iterdir():
    currentStrain = strainDict[assemblyFolder.name]
    for file in assemblyFolder.iterdir():
        if file.suffix == '.fna':
            genes = parseFasta(file)
            cpn60Found = False
            for gene in genes:
                if gene.lower().find('groel') != -1 or gene.lower().find('grol') != -1 or gene.lower().find('60 kda chaperonin') != -1:
                    cpn60Found = True
                    currentStrain.cpn60 = gene[gene.find('\n') + 1:].replace('\n', '')
            if not cpn60Found:
                print(currentStrain.id)
        elif file.suffix == '.faa':
            proteins = parseFasta(file)
            for prot in proteins:
                newProt = GlycanAnalysisProtein(prot)
                if not newProt.sequence in proteinDict.keys():
                    proteinDict[newProt.sequence] = newProt
                proteinDict[newProt.sequence].addTaxa(currentStrain.id)
                currentStrain.aaSeqs.add(newProt.sequence)

In [16]:
# Write out deduplicated proteins
#toWrite = []
#for prot in proteinDict.values():
#    toWrite.append(prot.getEntry())
#with analysisPath.joinpath('gardnerella/deduplicated_gardnerella_proteins.fasta').open(mode='w', newline='') as outfile:
#    outfile.write(''.join(toWrite))

In [17]:
# Write out all the cpn60 sequences
#toWrite = []
#for strain in strainDict.values():
#    if strain.cpn60 == '':
#        print(strain.id)
#    name = strain.name.replace(' ', '_')
#    toWrite.append(f'>{name}_{strain.id.replace(" ", "_")} cpn60\n{strain.cpn60}\n')
#with analysisPath.joinpath('gardnerella/cpn60_sequences.fasta').open(mode='w', newline='') as outfile:
#    outfile.write(''.join(toWrite))

In [18]:
# I made the Gardnerella tree and organized them. Now I'll read back in the order in which the isolates appear on the tree
order = []
with analysisPath.joinpath('gardnerella/tree_order.csv').open(mode='r', encoding='utf-8-sig') as infile:
    reader = csv.reader(infile)
    for row in reader:
        order.append(row[0])

In [19]:
# I performed a BLASTp searches to identify homologs of metabolic proteins in the Gardnerella. Load in those lists.
filesDict = {
    'nanA':analysisPath.joinpath('gardnerella/homologs/homologs_nanA.csv'),
    'nanK':analysisPath.joinpath('gardnerella/homologs/homologs_nanK.csv'),
    'nanE':analysisPath.joinpath('gardnerella/homologs/homologs_nanE.csv'),
    'nagA':analysisPath.joinpath('gardnerella/homologs/homologs_nagA.csv'),
    'nagB':analysisPath.joinpath('gardnerella/homologs/homologs_nagB.csv'),
    'nagE':analysisPath.joinpath('gardnerella/homologs/homologs_nagE.csv'),
    'nagK':analysisPath.joinpath('gardnerella/homologs/homologs_nagK.csv'),
    'galK':analysisPath.joinpath('gardnerella/homologs/homologs_galK.csv'),
    'galT':analysisPath.joinpath('gardnerella/homologs/homologs_galT.csv'),
    'galE':analysisPath.joinpath('gardnerella/homologs/homologs_galE.csv'),
    'pgmA':analysisPath.joinpath('gardnerella/homologs/homologs_pgmA.csv'),
    'agaK':analysisPath.joinpath('gardnerella/homologs/homologs_agaK.csv'),
    'agaD':analysisPath.joinpath('gardnerella/homologs/homologs_agaD.csv'),
    'agaA':analysisPath.joinpath('gardnerella/homologs/homologs_agaA.csv'),
    'agaS':analysisPath.joinpath('gardnerella/homologs/homologs_agaS.csv'),
    'pfkA':analysisPath.joinpath('gardnerella/homologs/homologs_pfkA.csv'),
    'kbaY':analysisPath.joinpath('gardnerella/homologs/homologs_kbaY.csv'),
    'fucA1':analysisPath.joinpath('gardnerella/homologs/homologs_fucA.csv'),
    'fucB':analysisPath.joinpath('gardnerella/homologs/homologs_fucB.csv'),
    'fucD':analysisPath.joinpath('gardnerella/homologs/homologs_fucD.csv'),
    'fucH':analysisPath.joinpath('gardnerella/homologs/homologs_fucH.csv'),
    'fucO':analysisPath.joinpath('gardnerella/homologs/homologs_fucO.csv')
}
homologsDict = {} #key=protein name, value=list of homologous protein IDs
for proteinName, filePath in filesDict.items():
    homologsDict[proteinName] = loadHomologs(filePath)
homologsDict = combineHomologs(homologsDict, 'nagE/K', ['nagE', 'nagK'])
homologsDict = combineHomologs(homologsDict, 'agaD/K', ['agaD', 'agaK'])
protOrder = ['nanA', 'nanK', 'nanE', 'nagA', 'nagB', 'nagE/K', 'galK', 'galT', 'galE', 'pgmA', 'agaD/K', 'agaA', 'agaS', 'pfkA', 'kbaY', 'fucA1', 'fucB', 'fucD', 'fucH', 'fucO']

In [20]:
# Get the amino acid sequences of all the protein homologs so I can look for them in the Gardnerella strains
aaDict = {}
for proteinName in homologsDict.keys():
    aaDict[proteinName] = set()
for protein in proteinDict.values():
    for key, homologList in homologsDict.items():
        if protein.id in homologsDict[key]:
            aaDict[key].add(protein.sequence)

In [21]:
# Get whether the isolates have one of each homolog, in the tree order
homologStatus = {'species':[], 'strain':[]}
for proteinName in protOrder:
    homologStatus[proteinName] = []
for strain in order:
    for isolate in strainDict.values():
        if isolate.id == strain:
            listForStrain = generateEmptyForSupp(isolate.id)
            homologStatus['strain'].append(strain)
            homologStatus['species'].append(isolate.name)
            for proteinName in homologsDict.keys():
                protPresent = '-'
                for seq in aaDict[proteinName]:
                    if seq in isolate.aaSeqs:
                        protPresent = '+'
                        listForStrain[fullProtList.index(proteinName)] = proteinDict[seq].id
                homologStatus[proteinName].append(protPresent)
            suppDataHolder.append(listForStrain)

In [27]:
pd.DataFrame(homologStatus).to_csv(analysisPath.joinpath('gardnerella/gardnerella_homolog_summary_v2.csv'), index=False)

In [22]:
### Repeat the analysis for Prevotella
# First, read through the metadata file and initialize all of the Gardnerella strains
strainDict = {} #key=Accession number, value=Strain object
metadata = pd.read_csv(analysisPath.joinpath('prevotella/prevotella_metadata.csv'))
for index, row in metadata.iterrows():
    strainDict[row['Assembly Accession']] = Strain(row)

In [23]:
# Go through all the sequences for each strain. Pull out all of the proteins so I can make a BLAST-able database that retains strain information, and retrieve the cpn60 sequence for each strain.
proteinDict = {} # key=amino acid sequence, value=protein object
for assemblyFolder in analysisPath.joinpath('prevotella/sequence_data/').iterdir():
    currentStrain = strainDict[assemblyFolder.name]
    for file in assemblyFolder.iterdir():
        if file.suffix == '.fna':
            genes = parseFasta(file)
            cpn60Found = False
            for gene in genes:
                if gene.lower().find('groel') != -1 or gene.lower().find('grol') != -1 or gene.lower().find('60 kda chaperonin') != -1:
                    cpn60Found = True
                    currentStrain.cpn60 = gene[gene.find('\n') + 1:].replace('\n', '')
            if not cpn60Found:
                print(f'{currentStrain.id}: {currentStrain.accession}')
        elif file.suffix == '.faa':
            proteins = parseFasta(file)
            for prot in proteins:
                newProt = GlycanAnalysisProtein(prot)
                if not newProt.sequence in proteinDict.keys():
                    proteinDict[newProt.sequence] = newProt
                proteinDict[newProt.sequence].addTaxa(currentStrain.id)
                currentStrain.aaSeqs.add(newProt.sequence)

In [24]:
# Write out deduplicated proteins
#toWrite = []
#for prot in proteinDict.values():
#    toWrite.append(prot.getEntry())
#with analysisPath.joinpath('prevotella/deduplicated_prevotella_proteins.fasta').open(mode='w', newline='') as outfile:
#    outfile.write(''.join(toWrite))

In [25]:
# Write out all the cpn60 sequences
#toWrite = []
#for strain in strainDict.values():
#    if strain.cpn60 == '':
#        print(strain.id)
#    name = strain.name.replace(' ', '_')
#    toWrite.append(f'>{name}_{strain.id.replace(" ", "_")} cpn60\n{strain.cpn60}\n')
#with analysisPath.joinpath('prevotella/prevotella_cpn60_sequences.fasta').open(mode='w', newline='') as outfile:
#    outfile.write(''.join(toWrite))

In [26]:
# I made the Prevotella tree and organized them. Now I'll read back in the order in which the isolates appear on the tree
order = []
with analysisPath.joinpath('prevotella/tree_order.csv').open(mode='r', encoding='utf-8-sig') as infile:
    reader = csv.reader(infile)
    for row in reader:
        order.append(row[0])

In [27]:
# I performed a BLASTp searches to identify homologs of metabolic proteins in the Prevotellaceae genomes. Load in those lists.
filesDict = {
    'nanA':analysisPath.joinpath('prevotella/homologs/homologs_nanA.csv'),
    'nanE-II':analysisPath.joinpath('prevotella/homologs/homologs_nanE2.csv'),
    'nagA':analysisPath.joinpath('prevotella/homologs/homologs_nagA.csv'),
    'nagB':analysisPath.joinpath('prevotella/homologs/homologs_nagB.csv'),
    'nagE':analysisPath.joinpath('prevotella/homologs/homologs_nagE.csv'),
    'nagK':analysisPath.joinpath('prevotella/homologs/homologs_nagK.csv'),
    'galK':analysisPath.joinpath('prevotella/homologs/homologs_galK.csv'),
    'galT':analysisPath.joinpath('prevotella/homologs/homologs_galT.csv'),
    'galE':analysisPath.joinpath('prevotella/homologs/homologs_galE.csv'),
    'pgmA':analysisPath.joinpath('prevotella/homologs/homologs_pgmA.csv'),
    'agaK':analysisPath.joinpath('prevotella/homologs/homologs_agaK.csv'),
    'agaD':analysisPath.joinpath('prevotella/homologs/homologs_agaD.csv'),
    'agaA':analysisPath.joinpath('prevotella/homologs/homologs_agaA.csv'),
    'agaS':analysisPath.joinpath('prevotella/homologs/homologs_agaS.csv'),
    'pfkA':analysisPath.joinpath('prevotella/homologs/homologs_pfkA.csv'),
    'kbaY':analysisPath.joinpath('prevotella/homologs/homologs_kbaY.csv'),
    'fucI':analysisPath.joinpath('prevotella/homologs/homologs_fucI.csv'),
    'fucK':analysisPath.joinpath('prevotella/homologs/homologs_fucK.csv'),
    'fucA':analysisPath.joinpath('prevotella/homologs/homologs_fucA.csv')
}
homologsDict = {} #key=protein name, value=list of homologous protein IDs
for proteinName, filePath in filesDict.items():
    homologsDict[proteinName] = loadHomologs(filePath)
homologsDict = combineHomologs(homologsDict, 'nagE/K', ['nagE', 'nagK'])
homologsDict = combineHomologs(homologsDict, 'agaD/K', ['agaD', 'agaK'])
protOrder = ['nanA', 'nanE-II', 'nagA', 'nagB', 'nagE/K', 'galK', 'galT', 'galE', 'pgmA', 'agaD/K', 'agaA', 'agaS', 'pfkA', 'kbaY', 'fucI', 'fucK', 'fucA']

In [28]:
# Get the amino acid sequences of all the protein homologs so I can look for them in the Prevotella strains
aaDict = {}
for proteinName in homologsDict.keys():
    aaDict[proteinName] = set()
for protein in proteinDict.values():
    for key, homologList in homologsDict.items():
        if protein.id in homologsDict[key]:
            aaDict[key].add(protein.sequence)

In [29]:
# Get whether the isolates have one of each homolog, in the tree order
homologStatus = {'species':[], 'strain':[]}
for proteinName in protOrder:
    homologStatus[proteinName] = []
for strain in order:
    for isolate in strainDict.values():
        if isolate.id == strain:
            listForStrain = generateEmptyForSupp(isolate.id)
            homologStatus['strain'].append(strain)
            homologStatus['species'].append(isolate.name)
            for proteinName in homologsDict.keys():
                protPresent = '-'
                for seq in aaDict[proteinName]:
                    if seq in isolate.aaSeqs:
                        protPresent = '+'
                        listForStrain[fullProtList.index(proteinName)] = proteinDict[seq].id
                homologStatus[proteinName].append(protPresent)
            suppDataHolder.append(listForStrain)

In [17]:
pd.DataFrame(homologStatus).to_csv(analysisPath.joinpath('prevotella/prevotella_homolog_summary_v2.csv'), index=False)

In [30]:
### Rerun the analysis for other species of Lactobacillus - iners, gasseri, & jensenii
strainDict = {} #key=Accession number, value=Strain object
metadata = pd.read_csv(analysisPath.joinpath('lactobacilli/lactobacilli_metadata.csv'))
for index, row in metadata.iterrows():
    strainDict[row['Assembly Accession']] = Strain(row)

In [31]:
# Go through all the sequences for each strain. Pull out all of the proteins so I can make a BLAST-able database that retains strain information, and retrieve the cpn60 sequence for each strain.
proteinDict = {} # key=amino acid sequence, value=protein object
for assemblyFolder in analysisPath.joinpath('lactobacilli/sequence_data/').iterdir():
    currentStrain = strainDict[assemblyFolder.name]
    for file in assemblyFolder.iterdir():
        if file.suffix == '.fna':
            genes = parseFasta(file)
            cpn60Found = False
            for gene in genes:
                if gene.lower().find('groel') != -1 or gene.lower().find('grol') != -1 or gene.lower().find('60 kda chaperonin') != -1:
                    cpn60Found = True
                    currentStrain.cpn60 = gene[gene.find('\n') + 1:].replace('\n', '')
            if not cpn60Found:
                print(currentStrain.id)
        elif file.suffix == '.faa':
            proteins = parseFasta(file)
            for prot in proteins:
                newProt = GlycanAnalysisProtein(prot)
                if not newProt.sequence in proteinDict.keys():
                    proteinDict[newProt.sequence] = newProt
                proteinDict[newProt.sequence].addTaxa(currentStrain.id)
                currentStrain.aaSeqs.add(newProt.sequence)

In [32]:
# Write out deduplicated proteins
#toWrite = []
#for prot in proteinDict.values():
#    toWrite.append(prot.getEntry())
#with analysisPath.joinpath('lactobacilli/deduplicated_lactobacillus_proteins.fasta').open(mode='w', newline='') as outfile:
#    outfile.write(''.join(toWrite))

In [33]:
# Write out all the cpn60 sequences
#toWrite = []
#for strain in strainDict.values():
#    toWrite.append(f'>{strain.id} cpn60\n{strain.cpn60}\n')
#with analysisPath.joinpath('lactobacilli/lactobacilli_cpn60_sequences.fasta').open(mode='w', newline='') as outfile:
#    outfile.write(''.join(toWrite))

In [34]:
# I made the Lactobacilli tree and organized them. Now I'll read back in the order in which the isolates appear on the tree
order = []
with analysisPath.joinpath('lactobacilli/tree_order.csv').open(mode='r', encoding='utf-8-sig') as infile:
    reader = csv.reader(infile)
    for row in reader:
        order.append(row[0])

In [35]:
# I performed a BLASTp searches to identify homologs of metabolic proteins in the Lactobacillus genomes. Load in those lists.
filesDict = {
    'nanA':analysisPath.joinpath('lactobacilli/homologs/homologs_nanA.csv'),
    'nanK':analysisPath.joinpath('lactobacilli/homologs/homologs_nanK.csv'),
    'nanE':analysisPath.joinpath('lactobacilli/homologs/homologs_nanE.csv'),
    'nagA':analysisPath.joinpath('lactobacilli/homologs/homologs_nagA.csv'),
    'nagB':analysisPath.joinpath('lactobacilli/homologs/homologs_nagB.csv'),
    'nagE':analysisPath.joinpath('lactobacilli/homologs/homologs_nagE.csv'),
    'nagK':analysisPath.joinpath('lactobacilli/homologs/homologs_nagK.csv'),
    'galK':analysisPath.joinpath('lactobacilli/homologs/homologs_galK.csv'),
    'galT':analysisPath.joinpath('lactobacilli/homologs/homologs_galT.csv'),
    'galE':analysisPath.joinpath('lactobacilli/homologs/homologs_galE.csv'),
    'pgmA':analysisPath.joinpath('lactobacilli/homologs/homologs_pgmA.csv'),
    'lacE':analysisPath.joinpath('lactobacilli/homologs/homologs_lacE.csv'),
    'lacB':analysisPath.joinpath('lactobacilli/homologs/homologs_lacB.csv'),
    'agaK':analysisPath.joinpath('lactobacilli/homologs/homologs_agaK.csv'),
    'agaD':analysisPath.joinpath('lactobacilli/homologs/homologs_agaD.csv'),
    'agaA':analysisPath.joinpath('lactobacilli/homologs/homologs_agaA.csv'),
    'agaS':analysisPath.joinpath('lactobacilli/homologs/homologs_agaS.csv'),
    'pfkA':analysisPath.joinpath('lactobacilli/homologs/homologs_pfkA.csv'),
    'kbaY':analysisPath.joinpath('lactobacilli/homologs/homologs_kbaY.csv'),
    'fucI':analysisPath.joinpath('lactobacilli/homologs/homologs_fucI.csv'),
    'fucK':analysisPath.joinpath('lactobacilli/homologs/homologs_fucK.csv'),
    'fucA':analysisPath.joinpath('lactobacilli/homologs/homologs_fucA.csv')
}
homologsDict = {} #key=protein name, value=list of homologous protein IDs
for proteinName, filePath in filesDict.items():
    homologsDict[proteinName] = loadHomologs(filePath)
homologsDict = combineHomologs(homologsDict, 'nagE/K', ['nagE', 'nagK'])
homologsDict = combineHomologs(homologsDict, 'agaD/K', ['agaD', 'agaK'])
protOrder = ['nanA', 'nanK', 'nanE', 'nagA', 'nagB', 'nagE/K', 'galK', 'galT', 'galE', 'pgmA', 'lacE', 'lacB', 'pfkA', 'kbaY', 'agaD/K', 'agaA', 'agaS', 'fucI', 'fucK', 'fucA']

In [36]:
# Get the amino acid sequences of all the protein homologs so I can look for them in the Lactobacillus strains
aaDict = {}
for proteinName in homologsDict.keys():
    aaDict[proteinName] = set()
for protein in proteinDict.values():
    for key, homologList in homologsDict.items():
        if protein.id in homologsDict[key]:
            aaDict[key].add(protein.sequence)

In [37]:
# Get whether the isolates have one of each homolog, in the tree order
homologStatus = {'species':[], 'strain':[]}
for proteinName in protOrder:
    homologStatus[proteinName] = []
for strain in order:
    for isolate in strainDict.values():
        if isolate.id == strain:
            listForStrain = generateEmptyForSupp(isolate.id)
            homologStatus['strain'].append(strain)
            homologStatus['species'].append(isolate.name)
            for proteinName in homologsDict.keys():
                protPresent = '-'
                for seq in aaDict[proteinName]:
                    if seq in isolate.aaSeqs:
                        protPresent = '+'
                        listForStrain[fullProtList.index(proteinName)] = proteinDict[seq].id
                homologStatus[proteinName].append(protPresent)
            suppDataHolder.append(listForStrain)

In [49]:
pd.DataFrame(homologStatus).to_csv(analysisPath.joinpath('lactobacilli/lactobacilli_homolog_summary_v2.csv'), index=False)

In [38]:
### Write out the IDs of each isolate's homologs to a csv file
with analysisPath.joinpath('glycan_metabolic_homologs.csv').open(mode='w', newline='', encoding='utf-8') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(fullProtList)
    for row in suppDataHolder:
        writer.writerow(row)